In [1]:
import numpy as np
import spectral as spy
import os
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from concurrent.futures import ProcessPoolExecutor, as_completed
from functools import partial # Nécessaire pour fixer les arguments
from tqdm import tqdm
import csv


# *** IMPORT CRUCIAL ***
from data_processor import process_single_image_and_save, compute_hist_features

In [2]:


def create_metadata_csv(base_dir, output_csv_file='hyperspectral_metadata.csv'):
    """
    Parcourt les dossiers de catégories (G1-G4), extrait les métadonnées de chaque image 
    et les sauvegarde dans un fichier CSV.
    """
    
    categories = ['G1', 'G2', 'G3', 'G4']
    data_dirs = [os.path.join(base_dir, cat) for cat in categories]
    
    metadata_list = []
    
    # En-têtes du fichier CSV
    fieldnames = ['Nom_Fichier_HDR', 'Classe', 'Lignes', 'Colonnes', 'Bandes']
    
    print(f"Extraction des métadonnées des dossiers : {categories}...")

    # Boucle sur les dossiers de catégories
    for category_path in data_dirs:
        class_name = os.path.basename(category_path) # Ex: 'G1'
        
        # Chercher tous les fichiers .hdr qui ne sont pas des références Dark/White
        image_files_hdr = glob(os.path.join(category_path, '**', '*.hdr'), recursive=True)
        image_files_hdr = [f for f in image_files_hdr if 'Dark' not in f and 'White' not in f and 'Calib' not in f and 'white' not in f]

        if not image_files_hdr:
            print(f"Avertissement : Aucune image de mesure trouvée dans {class_name}.")
            continue

        for hdr_file in image_files_hdr:
            try:
                # Utiliser spy.open_image (ou spy.envi.open) sans charger le .bin
                # L'objet Image généré par spy contient les métadonnées.
                img_metadata = spy.envi.open(hdr_file, hdr_file.replace('.hdr', '.bin')) 
                
                # Récupération des informations
                file_name_only = os.path.basename(hdr_file)
                # 3. CORRECTION : Accéder aux dimensions par la propriété .shape
                # La forme est (Lignes, Colonnes, Bandes)
                lines, samples, bands = img_metadata.shape
                
                metadata_list.append({
                    'Nom_Fichier_HDR': file_name_only,
                    'Classe': class_name,
                    'Lignes': lines,
                    'Colonnes': samples,
                    'Bandes': bands
                })
                
            except Exception as e:
                # Capture les erreurs si un fichier .hdr est mal formé ou si le .bin est introuvable (mais les metadata sont souvent lues quand même)
                print(f"Erreur de lecture des métadonnées pour {os.path.basename(hdr_file)}: {e}")

    # --- Écriture du fichier CSV ---
    try:
        with open(output_csv_file, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(metadata_list)
            
        print(f"\nSUCCESS: Le fichier CSV de métadonnées a été créé : {output_csv_file}")
        print(f"Total de {len(metadata_list)} images documentées.")
        
    except Exception as e:
        print(f"ERREUR lors de l'écriture du fichier CSV : {e}")

In [3]:
# # BASE_DIR doit pointer vers le répertoire parent contenant G1, G2, G3, G4.
# BASE_DIR = '/Volumes/Elements/Framatome' 

# # L'exécution crée le fichier hyperspectral_metadata.csv
# create_metadata_csv(BASE_DIR, output_csv_file='hyperspectral_dataset_summary.csv')

In [4]:

def extract_class_from_filename(file_name):
    for cat in ['G1', 'G2', 'G3', 'G4']:
        if cat in file_name:
            return cat
    return 'Unknown'

In [5]:

LOG_FILE = 'processed_hdrs_log.txt'
DATASET_REFLEC_DIR = 'dataset_reflec_test'

# --- NOUVELLE VERSION DE create_dataset AVEC PARALLÉLISATION ---
def create_dataset_parallel(base_dir, max_workers=None): # max_workers par défaut aux cœurs du CPU
    # ... (le début du code reste le même, y compris la recherche des références Dark/White) ...
    
    L_min = 708
    # 1. PRÉPARATION ET CRÉATION DU DOSSIER CIBLE
    os.makedirs(DATASET_REFLEC_DIR, exist_ok=True) # Crée le dossier s'il n'existe pas

    # Définir le mapping des catégories (labels numériques)
    categories = ['G1', 'G2', 'G3', 'G4']
    label_map = {cat: i for i, cat in enumerate(categories)}
    
    # Dossiers contenant les images hyperspectrales
    data_dirs = [os.path.join(base_dir, cat) for cat in categories]
    # Liste pour stocker les tâches à exécuter
    tasks = [] 
    
    # Boucle sur les catégories pour générer les tâches (sans les exécuter)
    # for category_path in data_dirs:
    #     # 1. Résoudre le chemin réel du fichier DARK/WHITE (références spécifiques à la catégorie)
    #     # ... (glob pour dark_hdr et white_hdr, puis vérification de l'existence)
    #     # 1. Résoudre le chemin réel du fichier DARK
    #     # chaque catégorie a son propre fichier de référence Dark/White CalibFin ou CalibEnd

    #     # Motif général pour les fichiers contenant 'Dark' et l'extension .hdr
    #     dark_pattern = os.path.join(category_path, '**', '*Dark*.hdr')
    #     # Motif général pour les fichiers contenant 'White' et l'extension .hdr
    #     white_pattern = os.path.join(category_path, '**', '*White*.hdr')

    #     # --- 2. Recherche et Filtrage ---
    
    #     # On cherche tous les Dark/White
    #     dark_candidates = glob(dark_pattern, recursive=True)
    #     white_candidates = glob(white_pattern, recursive=True)

    #     # Filtrage : On ne retient que ceux qui contiennent le mot 'Calib' pour s'assurer que ce sont des références.
    #     # On s'assure aussi de ne pas avoir de duplication.
    #     dark_ref_list = [f for f in dark_candidates if 'Calib' in f or 'calib' in f]
    #     white_ref_list = [f for f in white_candidates if 'Calib' in f or 'calib' in f]
        
    #     # Enlève les doublons si glob trouve des chemins multiples pour le même fichier (peu probable, mais sécurisant)
    #     dark_ref_list = list(set(dark_ref_list))
    #     white_ref_list = list(set(white_ref_list))

    #     # Vérifier et extraire le chemin
    #     if not dark_ref_list or not white_ref_list:
    #         print(f"Avertissement : Références CalibFin/CalibEnd Dark/White manquantes dans {category_path}. Catégorie ignorée.")
    #         continue
        
    #     dark_hdr = dark_ref_list[0]
    #     white_hdr = white_ref_list[0]
    #     label = label_map.get(os.path.basename(category_path), -1)

    #     # 2. Chercher les images de mesure
    #     image_files_hdr = glob(os.path.join(category_path, '**', '*.hdr'), recursive=True)
    #     image_files_hdr = [f for f in image_files_hdr if 'Dark' not in f and 'White' not in f and 'white' not in f and 'Calib' not in f and 'calib' not in f]
        
    #     # 3. Créer une tâche pour chaque image de mesure
    #     for hdr_file in image_files_hdr:
    #         tasks.append({
    #             'hdr': hdr_file, 
    #             'dark': dark_hdr, 
    #             'white': white_hdr, 
    #             'label': label
    #         })

    # --- Recherche des Références (Dark/White) - Logique de recherche robuste ---
    dark_candidates = glob(os.path.join(base_dir, '**', '*Dark*.hdr'), recursive=True)
    white_candidates = glob(os.path.join(base_dir, '**', '*White*.hdr'), recursive=True)

    dark_ref_list = [f for f in dark_candidates if 'Calib' in f or 'calib' in f]
    white_ref_list = [f for f in white_candidates if 'Calib' in f or 'calib' in f]
    
    dark_ref_list = list(set(dark_ref_list))
    white_ref_list = list(set(white_ref_list))

    if not dark_ref_list or not white_ref_list:
        print(f"Avertissement : Références Dark/White (Calib) manquantes dans {base_dir}. Catégorie ignorée.")
        

    dark_hdr = dark_ref_list[0]
    white_hdr = white_ref_list[0]

    image_files_hdr = glob(os.path.join(base_dir, '**', '*.hdr'), recursive=True)
    image_files_hdr = [f for f in image_files_hdr if 'Dark' not in f and 'White' not in f and 'Calib' not in f and 'calib' not in f and 'white' not in f]

    for hdr_file in image_files_hdr:
        class_name = extract_class_from_filename(hdr_file)
        label = label_map.get(class_name, -1)
        tasks.append({
            'hdr': hdr_file, 
            'dark': dark_hdr, 
            'white': white_hdr, 
            'label': label
        }) 

    # --- GESTION DE LA REPRISE (LOG des fichiers traités) ---
    processed_hdrs = set()
    if os.path.exists(LOG_FILE):
        with open(LOG_FILE, 'r') as f:
            processed_hdrs = set(f.read().splitlines())
    
    # Filtrer les tâches pour ne garder que les fichiers non traités
    tasks_to_process = [t for t in tasks if t['hdr'] not in processed_hdrs]
    
    print(f"Total de fichiers à traiter (restants) : {len(tasks_to_process)} / {len(tasks)}.")

    # --- EXÉCUTION PARALLÈLE ET SAUVEGARDE INDIVIDUELLE ---
    
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        # Soumettre toutes les tâches restantes. Le résultat attendu est le cube de réflectance.
        futures = {executor.submit(process_single_image_and_save, 
                                # Arguments existants
                                t['hdr'], t['dark'], t['white'], t['label'], 
                                # NOUVEL ARGUMENT
                                L_min): t['hdr'] 
               for t in tasks_to_process}
        
        # Boucle sur les résultats COMPLÉTÉS (as_completed)
        for future in tqdm(as_completed(futures), total=len(tasks_to_process), desc="Calibration et Sauvegarde NPY"):
            
            hdr_path = futures[future]
            
            try:
                # La fonction process_single_image_and_save retourne None ou un booléen de succès
                future.result() 
                
                # Ajouter le fichier au log uniquement en cas de succès
                with open(LOG_FILE, 'a') as f:
                    f.write(f"{hdr_path}\n")
                    
            except Exception as e:
                 print(f"\n[ERREUR FATALE WORKER] sur {hdr_path}. Le processus a planté ou la sauvegarde a échoué: {e}")
                 # Le fichier n'est pas ajouté au log, il sera retraité à la prochaine exécution.
                 continue

    print("\nFIN DE L'EXTRACTION. Tous les cubes de réflectance sont sauvegardés au format NPY.")

    

In [6]:
# --- 3. CHARGER LE DATASET ---
# Code à exécuter dans votre environnement principal

# if __name__ == '__main__':
#     # Le reste de votre code de chargement ici
#     BASE_DIR = '/Volumes/Elements/Framatome/Echantillons cibles' # Assurez-vous que ceci est défini!
    
#     # L'appel à la fonction est maintenant protégé
#     create_dataset_parallel(BASE_DIR, max_workers=7) # Vous pouvez ajuster max_workers selon vos besoins
    

In [7]:
# Constantes globales nécessaires à l'ensemble du script
L_min = 708      # Nombre de lignes minimum (pour le redimensionnement spatial)
BAND_START = 30  # Indice de bande pour 410 nm
BAND_END = 465   # Indice de bande pour couper l'extrême fin (avant 1010 nm)
DATASET_HIST_DIR = 'dataset_hist2'
LOG_FILE = 'ghost_hist_processed_log.txt' # Nouveau log pour cette étape
CHECKPOINT_FILE = 'ghost_hist_checkpoint.npz' # Nouveau checkpoint

In [8]:


def create_ghost_dataset(base_dir, max_workers=None): 
    
    # 1. PRÉPARATION INITIALE DES TÂCHES
    categories = ['G1', 'G2', 'G3', 'G4']
    label_map = {cat: i for i, cat in enumerate(categories)}
    # data_dirs = [os.path.join(base_dir, cat) for cat in categories]
    tasks = [] 
    
    # for category_path in data_dirs:
    #     # --- Recherche des Références (Dark/White) - Logique de recherche robuste ---
    #     dark_candidates = glob(os.path.join(category_path, '**', '*Dark*.hdr'), recursive=True)
    #     white_candidates = glob(os.path.join(category_path, '**', '*White*.hdr'), recursive=True)

    #     dark_ref_list = [f for f in dark_candidates if 'Calib' in f or 'calib' in f]
    #     white_ref_list = [f for f in white_candidates if 'Calib' in f or 'calib' in f]
        
    #     dark_ref_list = list(set(dark_ref_list))
    #     white_ref_list = list(set(white_ref_list))

    #     if not dark_ref_list or not white_ref_list:
    #         print(f"Avertissement : Références Dark/White (Calib) manquantes dans {category_path}. Catégorie ignorée.")
    #         continue
          
    #     label = label_map.get(os.path.basename(category_path), -1)

    #     # 2. Chercher les images de mesure
    #     image_files_hdr = glob(os.path.join(category_path, '**', '*.hdr'), recursive=True)
    #     image_files_hdr = [f for f in image_files_hdr if 'Dark' not in f and 'White' not in f and 'Calib' not in f and 'calib' not in f and 'white' not in f]
        
    #     # 3. Créer une tâche pour chaque image de mesure
    #     for hdr_file in image_files_hdr:
    #         tasks.append({
    #             'hdr': hdr_file, 
    #             'label': label
    #         })

    # --- Recherche des Références (Dark/White) - Logique de recherche robuste ---
    # dark_candidates = glob(os.path.join(base_dir, '**', '*Dark*.hdr'), recursive=True)
    # white_candidates = glob(os.path.join(base_dir, '**', '*White*.hdr'), recursive=True)

    # dark_ref_list = [f for f in dark_candidates if 'Calib' in f or 'calib' in f]
    # white_ref_list = [f for f in white_candidates if 'Calib' in f or 'calib' in f]
    
    # dark_ref_list = list(set(dark_ref_list))
    # white_ref_list = list(set(white_ref_list))

    # if not dark_ref_list or not white_ref_list:
    #     print(f"Avertissement : Références Dark/White (Calib) manquantes dans {base_dir}. Catégorie ignorée.")
        

    image_files_hdr = glob(os.path.join(base_dir, '**', '*.npy'), recursive=True)
    # image_files_hdr = [f for f in image_files_hdr if 'Dark' not in f and 'White' not in f and 'Calib' not in f and 'calib' not in f and 'white' not in f]

    for npy_file in image_files_hdr:
        class_name = extract_class_from_filename(npy_file)
        label = label_map.get(class_name, -1)
        tasks.append({
            'npy': npy_file, 
            'label': label
        })  
    

    # 1. PRÉPARATION ET CRÉATION DU DOSSIER CIBLE
    os.makedirs(DATASET_HIST_DIR, exist_ok=True)
    
    # 2. GESTION DE LA REPRISE (LOG des fichiers traités)
    processed_hdrs = set()
    if os.path.exists(LOG_FILE):
        with open(LOG_FILE, 'r') as f:
            processed_hdrs = set(f.read().splitlines())
    
    # Filtrer les tâches pour ne garder que les fichiers non traités
    tasks_to_process = [t for t in tasks if t['npy'] not in processed_hdrs]

    # --- EXÉCUTION PARALLÈLE ET SAUVEGARDE INDIVIDUELLE ---
    print(f"Démarrage de l'extraction GHOST en parallèle sur {len(tasks_to_process)} fichiers...")

    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        
        futures = {executor.submit(
            compute_hist_features, 
            t['npy'], t['label']): t['npy'] for t in tasks_to_process}
        
        # Boucle sur les résultats COMPLÉTÉS
        for future in tqdm(as_completed(futures), total=len(tasks_to_process), desc="Extraction GHOST et Sauvegarde"):
            
            npy_path = futures[future]
            
            try:
                # Le résultat est un booléen (True pour succès)
                future.result() 
                
                # Ajouter le fichier au log uniquement en cas de succès
                with open(LOG_FILE, 'a') as f:
                    f.write(f"{npy_path}\n")
                    
            except Exception as e:
                 print(f"\n[ERREUR FATALE WORKER] sur {npy_path}. Le processus a planté ou la sauvegarde a échoué: {e}")
                 continue

    print("\nFIN DE L'EXTRACTION GHOST. Tous les histogrammes sont sauvegardés individuellement.")

In [9]:
if __name__ == '__main__':
    # Le reste de votre code de chargement ici
    BASE_DIR = 'dataset_reflec' # Assurez-vous que ceci est défini!
    
    # L'appel à la fonction est maintenant protégé
    create_ghost_dataset(BASE_DIR, max_workers=6) 

Démarrage de l'extraction GHOST en parallèle sur 42 fichiers...


Extraction GHOST et Sauvegarde:   0%|          | 0/42 [00:00<?, ?it/s]

(708, 1000, 435)
(708, 1000, 435)
(708, 1000)
(708, 1000)
0.0045459177
9.821078


Extraction GHOST et Sauvegarde:   2%|▏         | 1/42 [32:21<22:06:22, 1941.04s/it]

(708, 1000, 435)
(708, 1000, 435)
(708, 1000)
(708, 1000)
0.0021350868
8.82599
(708, 1000, 435)
(708, 1000, 435)
(708, 1000, 435)
(708, 1000, 435)
(708, 1000)
(708, 1000)
0.0022824162
1.3443063
(708, 1000)
(708, 1000)
0.002006473
1.1204816


...

(708, 1000, 435)
(708, 1000, 435)
(708, 1000)
(708, 1000)
0.0037955882
10.585228


Extraction GHOST et Sauvegarde:  90%|█████████ | 38/42 [1:13:33<02:47, 41.89s/it]

(708, 1000, 435)
(708, 1000, 435)
(708, 1000)
(708, 1000)
0.0020281973
1.2270564
(708, 1000, 435)
(708, 1000, 435)
(708, 1000)
(708, 1000)
0.0021886837
2.9969616
(708, 1000, 435)
(708, 1000, 435)
(708, 1000)
(708, 1000)
0.0019319281
1.8136226
(708, 1000, 435)
(708, 1000, 435)
(708, 1000)
(708, 1000)
0.0019878505
0.6545454


Extraction GHOST et Sauvegarde: 100%|██████████| 42/42 [1:16:15<00:00, 108.94s/it]



FIN DE L'EXTRACTION GHOST. Tous les histogrammes sont sauvegardés individuellement.


In [ ]:
# Utiliser mathplotlib pour visualiser quelques histogrammes extraits
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import numpy as np
import os
from glob import glob

def visualiser_histogrammes_echantillons(hist_dir, nb_echantillons=5):
    """
    Visualise des histogrammes aléatoires à partir d'un répertoire
    
    Paramètres:
        hist_dir: Chemin du répertoire contenant les histogrammes
        nb_echantillons: Nombre d'échantillons à visualiser
    """
    # Obtenir tous les fichiers .npy dans le répertoire
    fichiers_hist = glob(os.path.join(hist_dir, '*.npy'))
    
    if len(fichiers_hist) == 0:
        print("Aucun fichier d'histogramme trouvé.")
        return
    
    print(f"Trouvé {len(fichiers_hist)} fichiers d'histogrammes")
    
    # Sélection aléatoire d'échantillons
    echantillons_fichiers = np.random.choice(
        fichiers_hist, 
        size=min(nb_echantillons, len(fichiers_hist)), 
        replace=False
    )
    
    for fichier_hist in echantillons_fichiers:
        try:
            # Charger les données
            donnees = np.load(fichier_hist, allow_pickle=True)
            
            # Vérifier la structure des données
            if isinstance(donnees, np.ndarray) and donnees.dtype == object:
                # Si c'est un tableau d'objets numpy, prendre le premier élément
                hist_donnees = donnees.item() if donnees.size == 1 else donnees[()]
            else:
                hist_donnees = donnees.item() if donnees.ndim == 0 else donnees
            
            # Extraire l'histogramme et le label
            if isinstance(hist_donnees, dict):
                histogramme = hist_donnees['histogram']
                label = hist_donnees['label']
            else:
                # Si ce n'est pas un dictionnaire, adapter selon votre structure
                print(f"Structure inattendue dans {os.path.basename(fichier_hist)}")
                print(f"Type: {type(hist_donnees)}, Forme: {hist_donnees.shape if hasattr(hist_donnees, 'shape') else 'N/A'}")
                continue
            
            # Créer la visualisation
            plt.figure(figsize=(10, 5))
            plt.bar(range(len(histogramme)), histogramme, alpha=0.7, color='skyblue', edgecolor='black')
            plt.title(f'Histogramme GHOST\nFichier: {os.path.basename(fichier_hist)}\nClasse: {label}', fontsize=14)
            plt.xlabel('Bins', fontsize=12)
            plt.ylabel('Fréquence', fontsize=12)
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
            
            # Afficher quelques informations statistiques
            print(f"\nInformations pour {os.path.basename(fichier_hist)}:")
            print(f"  - Classe: {label}")
            print(f"  - Taille de l'histogramme: {len(histogramme)} bins")
            print(f"  - Valeur maximale: {np.max(histogramme):.2f}")
            print(f"  - Valeur moyenne: {np.mean(histogramme):.2f}")
            print(f"  - Somme totale: {np.sum(histogramme):.2f}")
            print("-" * 50)
            
        except Exception as e:
            print(f"Erreur lors du traitement de {os.path.basename(fichier_hist)}: {e}")
            continue




In [ ]:
visualiser_histogrammes_echantillons("dataset_hist2", nb_echantillons=3)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from IPython.display import display
from PIL import Image

def visualize_sampling_notebook(cube, center_y, center_x, object_positions, 
                                background_positions, title="Échantillonnage"):
    """
    Visualise l'échantillonnage dans un notebook Jupyter.
    
    Args:
        cube: Cube hyperspectral (H, W, Bands)
        center_y, center_x: Coordonnées du centre
        object_positions: Array de positions objet (N, 2)
        background_positions: Array de positions fond (M, 2)
        title: Titre du graphique
    """
    h, w, bands = cube.shape
    
    # === 1. CRÉATION DE L'IMAGE RGB ===
    if bands >= 3:
        # Sélectionner 3 bandes pour créer une image RGB
        # Bandes équi-réparties : début, milieu, fin
        band_r = bands // 4          # ~25%
        band_g = bands // 2          # ~50%
        band_b = 3 * bands // 4      # ~75%
        
        rgb_image = cube[:, :, [band_r, band_g, band_b]]
        print(f"Bandes RGB sélectionnées: R={band_r}, G={band_g}, B={band_b}")
    else:
        # Si moins de 3 bandes, grayscale répété
        rgb_image = np.repeat(cube[:, :, 0:1], 3, axis=2)
    
    # Normalisation [0, 255]
    rgb_min = rgb_image.min()
    rgb_max = rgb_image.max()
    rgb_image = ((rgb_image - rgb_min) / (rgb_max - rgb_min + 1e-8) * 255).astype(np.uint8)
    
    # === 2. CRÉATION DE LA FIGURE ===
    fig, axes = plt.subplots(2, 2, figsize=(16, 14))
    
    # --- Subplot 1: Image originale ---
    axes[0, 0].imshow(rgb_image)
    axes[0, 0].set_title('Image Originale (Pseudo-RGB)', fontsize=14, fontweight='bold')
    axes[0, 0].axis('off')
    
    # --- Subplot 2: Pixels objet (verts) ---
    overlay_object = rgb_image.copy()
    for pos in object_positions[::5]:  # Sous-échantillonner pour visibilité
        y, x = pos
        cv2.circle(overlay_object, (x, y), 2, (0, 255, 0), -1)
    
    # Marquer le centre en bleu
    cv2.circle(overlay_object, (center_x, center_y), 8, (0, 0, 255), -1)
    cv2.circle(overlay_object, (center_x, center_y), 10, (255, 255, 255), 2)
    
    axes[0, 1].imshow(overlay_object)
    axes[0, 1].set_title(f'Pixels Objet (Vert, n={len(object_positions)})\nCentre: ({center_y}, {center_x})', 
                        fontsize=14, fontweight='bold')
    axes[0, 1].axis('off')
    
    # --- Subplot 3: Pixels fond (rouges) ---
    overlay_background = rgb_image.copy()
    for pos in background_positions[::5]:
        y, x = pos
        cv2.circle(overlay_background, (x, y), 2, (255, 0, 0), -1)
    
    axes[1, 0].imshow(overlay_background)
    axes[1, 0].set_title(f'Pixels Fond Blanc (Rouge, n={len(background_positions)})', 
                        fontsize=14, fontweight='bold')
    axes[1, 0].axis('off')
    
    # --- Subplot 4: Tout combiné ---
    overlay_combined = rgb_image.copy()
    
    # Pixels fond en rouge
    for pos in background_positions[::5]:
        y, x = pos
        cv2.circle(overlay_combined, (x, y), 2, (255, 0, 0), -1)
    
    # Pixels objet en vert (par-dessus)
    for pos in object_positions[::5]:
        y, x = pos
        cv2.circle(overlay_combined, (x, y), 2, (0, 255, 0), -1)
    
    # Centre en bleu
    cv2.circle(overlay_combined, (center_x, center_y), 8, (0, 0, 255), -1)
    cv2.circle(overlay_combined, (center_x, center_y), 10, (255, 255, 255), 2)
    
    # Blend avec l'image originale pour meilleure visibilité
    overlay_combined = cv2.addWeighted(rgb_image, 0.6, overlay_combined, 0.4, 0)
    
    axes[1, 1].imshow(overlay_combined)
    axes[1, 1].set_title('Combiné: Vert=Objet, Rouge=Fond, Bleu=Centre', 
                        fontsize=14, fontweight='bold')
    axes[1, 1].axis('off')
    
    plt.suptitle(title, fontsize=16, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.show()
    
    # === 3. STATISTIQUES ===
    print(f"\n{'='*70}")
    print(f"STATISTIQUES DE L'ÉCHANTILLONNAGE")
    print(f"{'='*70}")
    print(f"Dimensions image: {h} × {w} × {bands} bandes")
    print(f"Centre détecté: ({center_y}, {center_x})")
    print(f"Pixels objet: {len(object_positions)}")
    print(f"Pixels fond: {len(background_positions)}")
    print(f"Total pixels: {len(object_positions) + len(background_positions)}")
    print(f"Ratio Objet/Fond: {len(object_positions) / len(background_positions):.2f}:1")


def test_sampling_on_file(file_path, n_pixels_object=10000, n_pixels_background=5000):
    """
    Teste l'échantillonnage sur un fichier et visualise le résultat.
    
    Usage dans notebook:
        test_sampling_on_file('dataset_reflec/image_G1_sample1.npy')
    """
    print(f"Chargement: {file_path}")
    
    # Charger le cube
    cube = np.load(file_path)
    
    if cube.ndim != 3:
        print(f"❌ Erreur: le fichier doit être 3D, reçu shape={cube.shape}")
        return
    
    print(f"Shape: {cube.shape}")
    
    # 1. Trouver le centre
    from cnn_1d import find_object_center, extract_pixels_around_center, detect_background_pixels
    
    print("\n1️⃣ Détection du centre de l'objet...")
    center_y, center_x, object_mask = find_object_center(cube, method='variance')
    print(f"   Centre trouvé: ({center_y}, {center_x})")
    
    # 2. Extraire pixels objet
    print(f"\n2️⃣ Extraction de {n_pixels_object} pixels objet autour du centre...")
    object_pixels, object_positions = extract_pixels_around_center(
        cube, center_y, center_x, n_pixels_object, method='circular'
    )
    print(f"   Pixels extraits: {len(object_pixels)}")
    
    # 3. Extraire pixels fond
    print(f"\n3️⃣ Extraction de {n_pixels_background} pixels de fond blanc...")
    background_pixels, background_positions = detect_background_pixels(
        cube, object_mask, n_pixels_background
    )
    print(f"   Pixels extraits: {len(background_pixels)}")
    
    # 4. Visualiser
    print(f"\n4️⃣ Visualisation...")
    visualize_sampling_notebook(
        cube, center_y, center_x, object_positions, background_positions,
        title=f"Échantillonnage - {os.path.basename(file_path)}"
    )
    
    # 5. Analyser les spectres
    print(f"\n5️⃣ Analyse des spectres moyens...")
    analyze_spectra(object_pixels, background_pixels, cube.shape[2])
    
    return object_pixels, background_pixels, object_positions, background_positions


def analyze_spectra(object_pixels, background_pixels, n_bands):
    """
    Analyse et compare les spectres moyens objet vs fond.
    """
    # Spectres moyens
    mean_object = np.mean(object_pixels, axis=0)
    mean_background = np.mean(background_pixels, axis=0)
    
    # Écarts-types
    std_object = np.std(object_pixels, axis=0)
    std_background = np.std(background_pixels, axis=0)
    
    # Visualisation
    fig, axes = plt.subplots(1, 2, figsize=(16, 5))
    
    bands = np.arange(n_bands)
    
    # --- Spectres moyens ---
    axes[0].plot(bands, mean_object, 'g-', linewidth=2, label='Objet (moyenne)')
    axes[0].fill_between(bands, mean_object - std_object, mean_object + std_object, 
                         color='green', alpha=0.2, label='Objet (±1 std)')
    
    axes[0].plot(bands, mean_background, 'r-', linewidth=2, label='Fond (moyenne)')
    axes[0].fill_between(bands, mean_background - std_background, mean_background + std_background,
                         color='red', alpha=0.2, label='Fond (±1 std)')
    
    axes[0].set_xlabel('Bande spectrale', fontsize=12)
    axes[0].set_ylabel('Intensité (réflectance)', fontsize=12)
    axes[0].set_title('Spectres Moyens: Objet vs Fond', fontsize=14, fontweight='bold')
    axes[0].legend(fontsize=10)
    axes[0].grid(True, alpha=0.3)
    
    # --- Différence ---
    diff = mean_object - mean_background
    
    axes[1].plot(bands, diff, 'b-', linewidth=2)
    axes[1].axhline(y=0, color='k', linestyle='--', alpha=0.5)
    axes[1].fill_between(bands, 0, diff, where=(diff > 0), color='green', alpha=0.3, label='Objet > Fond')
    axes[1].fill_between(bands, 0, diff, where=(diff < 0), color='red', alpha=0.3, label='Fond > Objet')
    
    axes[1].set_xlabel('Bande spectrale', fontsize=12)
    axes[1].set_ylabel('Différence (Objet - Fond)', fontsize=12)
    axes[1].set_title('Différence Spectrale', fontsize=14, fontweight='bold')
    axes[1].legend(fontsize=10)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Statistiques
    print(f"\n{'='*70}")
    print(f"ANALYSE SPECTRALE")
    print(f"{'='*70}")
    print(f"Intensité moyenne objet: {mean_object.mean():.4f} ± {mean_object.std():.4f}")
    print(f"Intensité moyenne fond: {mean_background.mean():.4f} ± {mean_background.std():.4f}")
    print(f"Différence moyenne: {diff.mean():.4f}")
    print(f"Séparabilité (diff/std): {abs(diff.mean()) / (std_object.mean() + std_background.mean()):.4f}")

In [ ]:
# Dans votre Jupyter Notebook

# 1. Importer les fonctions
from cnn_1d import find_object_center, extract_pixels_around_center, detect_background_pixels

# 2. Tester sur UN fichier
test_sampling_on_file('dataset_reflec/2025-05-18-G2_20_class1.npy', 
                      n_pixels_object=10000, 
                      n_pixels_background=5000)

# 3. Tester sur PLUSIEURS fichiers
files = ['dataset_reflec/2025-04-03-G1_00-1_class0.npy',
         'dataset_reflec/2025-05-18-G2_03_class1.npy',
         'dataset_reflec/2025-05-06-G3_16_class2.npy',
         'dataset_reflec/2025-05-18-G4_04_class3.npy']

for file in files:
    test_sampling_on_file(file)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from pathlib import Path
from tqdm import tqdm

def visualize_all_dataset_sampling(data_dir, n_pixels_object=10000, n_pixels_background=5000,
                                   save_dir='visualizations', max_images=None):
    """
    Visualise l'échantillonnage pour TOUTES les images du dataset.
    
    Args:
        data_dir: Répertoire contenant les fichiers .npy
        n_pixels_object: Nombre de pixels objet par image
        n_pixels_background: Nombre de pixels fond par image
        save_dir: Répertoire pour sauvegarder les visualisations
        max_images: Limite du nombre d'images (None = toutes)
    """
    # Import des fonctions nécessaires
    from cnn_1d import (find_object_center, extract_pixels_around_center, 
                        detect_background_pixels, extract_class_from_filename)
    
    # Créer le répertoire de sauvegarde
    os.makedirs(save_dir, exist_ok=True)
    
    # Lister tous les fichiers .npy
    files = sorted([f for f in os.listdir(data_dir) if f.endswith('.npy')])
    
    if max_images is not None:
        files = files[:max_images]
    
    print(f"{'='*70}")
    print(f"VISUALISATION DE {len(files)} IMAGES")
    print(f"{'='*70}")
    print(f"Pixels objet/image: {n_pixels_object}")
    print(f"Pixels fond/image: {n_pixels_background}")
    print(f"Sauvegarde dans: {save_dir}/")
    print(f"{'='*70}\n")
    
    # Statistiques globales
    all_stats = []
    
    # Boucle sur tous les fichiers
    for idx, filename in enumerate(tqdm(files, desc="Traitement des images")):
        file_path = os.path.join(data_dir, filename)
        
        try:
            # Charger le cube
            cube = np.load(file_path)
            
            if cube.ndim != 3:
                print(f"\n⚠️  Ignoré (pas 3D): {filename}")
                continue
            
            h, w, bands = cube.shape
            
            # Extraire la classe
            try:
                class_label = extract_class_from_filename(file_path)
            except:
                class_label = "Unknown"
            
            # 1. Détecter le centre
            center_y, center_x, object_mask = find_object_center(cube, method='variance')
            
            # 2. Extraire pixels objet
            object_pixels, object_positions = extract_pixels_around_center(
                cube, center_y, center_x, n_pixels_object, method='circular'
            )
            
            # 3. Extraire pixels fond
            background_pixels, background_positions = detect_background_pixels(
                cube, object_mask, n_pixels_background
            )
            
            # 4. Créer la visualisation
            fig = create_single_image_visualization(
                cube, center_y, center_x, object_positions, background_positions,
                class_label, filename, h, w, bands
            )
            
            # 5. Sauvegarder
            save_path = os.path.join(save_dir, f"{Path(filename).stem}_sampling.png")
            plt.savefig(save_path, dpi=200, bbox_inches='tight')
            plt.close(fig)
            
            # 6. Collecter statistiques
            stats = {
                'filename': filename,
                'class': class_label,
                'shape': (h, w, bands),
                'center': (center_y, center_x),
                'n_object': len(object_pixels),
                'n_background': len(background_pixels),
                'mean_object': object_pixels.mean(),
                'mean_background': background_pixels.mean(),
                'std_object': object_pixels.std(),
                'std_background': background_pixels.std()
            }
            all_stats.append(stats)
            
        except Exception as e:
            print(f"\n❌ Erreur pour {filename}: {e}")
            continue
    
    # Résumé final
    print(f"\n{'='*70}")
    print(f"RÉSUMÉ")
    print(f"{'='*70}")
    print(f"Images traitées: {len(all_stats)}/{len(files)}")
    print(f"Visualisations sauvegardées dans: {save_dir}/")
    print(f"{'='*70}\n")
    
    # Créer un résumé HTML
    create_html_summary(all_stats, save_dir)
    
    return all_stats


def create_single_image_visualization(cube, center_y, center_x, object_positions, 
                                      background_positions, class_label, filename,
                                      h, w, bands):
    """
    Crée une figure de visualisation pour UNE image.
    """
    # Créer image RGB
    if bands >= 3:
        band_r = bands // 4
        band_g = bands // 2
        band_b = 3 * bands // 4
        rgb_image = cube[:, :, [band_r, band_g, band_b]]
    else:
        rgb_image = np.repeat(cube[:, :, 0:1], 3, axis=2)
    
    # Normalisation
    rgb_image = ((rgb_image - rgb_image.min()) / 
                 (rgb_image.max() - rgb_image.min() + 1e-8) * 255).astype(np.uint8)
    
    # Créer figure avec 3 subplots
    fig = plt.figure(figsize=(18, 6))
    
    # --- Subplot 1: Image originale ---
    ax1 = plt.subplot(1, 3, 1)
    ax1.imshow(rgb_image)
    ax1.set_title(f'Image Originale\n{filename}\nClasse: {class_label}', 
                  fontsize=12, fontweight='bold')
    ax1.axis('off')
    
    # --- Subplot 2: Pixels objet (verts) ---
    ax2 = plt.subplot(1, 3, 2)
    overlay_object = rgb_image.copy()
    
    # Dessiner tous les pixels objet (sous-échantillonné pour visibilité)
    for pos in object_positions[::max(1, len(object_positions)//2000)]:
        y, x = pos
        cv2.circle(overlay_object, (x, y), 1, (0, 255, 0), -1)
    
    # Marquer le centre
    cv2.circle(overlay_object, (center_x, center_y), 6, (0, 0, 255), -1)
    cv2.circle(overlay_object, (center_x, center_y), 8, (255, 255, 255), 2)
    
    # Tracer le cercle de sélection
    radius = int(np.sqrt(len(object_positions) / np.pi))
    cv2.circle(overlay_object, (center_x, center_y), radius, (255, 255, 0), 2)
    
    ax2.imshow(overlay_object)
    ax2.set_title(f'Pixels Objet (n={len(object_positions)})\nCentre: ({center_y}, {center_x})\nRadius: {radius}', 
                  fontsize=12, fontweight='bold', color='green')
    ax2.axis('off')
    
    # --- Subplot 3: Pixels fond (rouges) ---
    ax3 = plt.subplot(1, 3, 3)
    overlay_background = rgb_image.copy()
    
    # Dessiner tous les pixels fond
    for pos in background_positions[::max(1, len(background_positions)//2000)]:
        y, x = pos
        cv2.circle(overlay_background, (x, y), 1, (255, 0, 0), -1)
    
    # Marquer les zones de bords préférées
    border_size = max(h, w) // 10
    cv2.rectangle(overlay_background, (0, 0), (border_size, h), (255, 255, 0), 2)
    cv2.rectangle(overlay_background, (w-border_size, 0), (w, h), (255, 255, 0), 2)
    cv2.rectangle(overlay_background, (0, 0), (w, border_size), (255, 255, 0), 2)
    cv2.rectangle(overlay_background, (0, h-border_size), (w, h), (255, 255, 0), 2)
    
    ax3.imshow(overlay_background)
    ax3.set_title(f'Pixels Fond (n={len(background_positions)})\nZones préférées: bords/coins', 
                  fontsize=12, fontweight='bold', color='red')
    ax3.axis('off')
    
    plt.suptitle(f'{filename} - Classe {class_label} - Shape: {h}×{w}×{bands}', 
                 fontsize=14, fontweight='bold', y=0.98)
    plt.tight_layout()
    
    return fig


def create_html_summary(all_stats, save_dir):
    """
    Crée un fichier HTML pour visualiser toutes les images facilement.
    """
    html_path = os.path.join(save_dir, 'index.html')
    
    # Calculer statistiques globales
    n_images = len(all_stats)
    classes = sorted(list(set([s['class'] for s in all_stats])))
    total_object = sum([s['n_object'] for s in all_stats])
    total_background = sum([s['n_background'] for s in all_stats])
    
    # Créer les lignes du tableau
    table_rows = []
    for s in all_stats:
        row = f"""
            <tr>
                <td>{s['filename']}</td>
                <td>{s['class']}</td>
                <td>{s['shape'][0]}×{s['shape'][1]}×{s['shape'][2]}</td>
                <td>({s['center'][0]}, {s['center'][1]})</td>
                <td>{s['n_object']}</td>
                <td>{s['n_background']}</td>
                <td>{s['mean_object']:.4f}</td>
                <td>{s['mean_background']:.4f}</td>
            </tr>
        """
        table_rows.append(row)
    
    # Créer les cartes d'images
    image_cards = []
    for s in all_stats:
        img_filename = f"{Path(s['filename']).stem}_sampling.png"
        card = f"""
        <div class="image-card">
            <img src="{img_filename}" alt="{s['filename']}">
            <div class="image-info">
                <strong>{s['filename']}</strong><br>
                Classe: {s['class']} | Shape: {s['shape'][0]}×{s['shape'][1]}×{s['shape'][2]}<br>
                Centre: ({s['center'][0]}, {s['center'][1]}) | 
                Objet: {s['n_object']} | Fond: {s['n_background']}
            </div>
        </div>
        """
        image_cards.append(card)
    
    # Template HTML (sans .format(), utilisation de f-string directement)
    html_content = f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Dataset Sampling Visualization</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 20px;
            background-color: #f5f5f5;
        }}
        h1 {{
            color: #333;
            text-align: center;
        }}
        .stats {{
            background: white;
            padding: 20px;
            margin: 20px 0;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }}
        .image-grid {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(600px, 1fr));
            gap: 20px;
            margin-top: 20px;
        }}
        .image-card {{
            background: white;
            padding: 15px;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }}
        .image-card img {{
            width: 100%;
            height: auto;
            border-radius: 4px;
        }}
        .image-info {{
            margin-top: 10px;
            font-size: 14px;
            color: #666;
        }}
        .image-info strong {{
            color: #333;
        }}
        table {{
            width: 100%;
            border-collapse: collapse;
            margin-top: 10px;
        }}
        th, td {{
            padding: 8px;
            text-align: left;
            border-bottom: 1px solid #ddd;
        }}
        th {{
            background-color: #4CAF50;
            color: white;
        }}
        tr:hover {{
            background-color: #f5f5f5;
        }}
    </style>
</head>
<body>
    <h1>🔬 Dataset Sampling Visualization</h1>
    
    <div class="stats">
        <h2>📊 Statistiques Globales</h2>
        <p><strong>Nombre d'images:</strong> {n_images}</p>
        <p><strong>Classes présentes:</strong> {', '.join(map(str, classes))}</p>
        <p><strong>Total pixels objet:</strong> {total_object:,}</p>
        <p><strong>Total pixels fond:</strong> {total_background:,}</p>
    </div>
    
    <div class="stats">
        <h2>📋 Détails par Image</h2>
        <table>
            <tr>
                <th>Fichier</th>
                <th>Classe</th>
                <th>Shape (H×W×B)</th>
                <th>Centre (Y, X)</th>
                <th>Pixels Objet</th>
                <th>Pixels Fond</th>
                <th>Mean Objet</th>
                <th>Mean Fond</th>
            </tr>
            {''.join(table_rows)}
        </table>
    </div>
    
    <h2>🖼️ Visualisations</h2>
    <div class="image-grid">
        {''.join(image_cards)}
    </div>
</body>
</html>
"""
    
    # Sauvegarder
    with open(html_path, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    print(f"\n✅ Résumé HTML créé: {html_path}")
    print(f"   Ouvrez ce fichier dans votre navigateur pour voir toutes les visualisations")

def visualize_dataset_summary(all_stats):
    """
    Créer une vue d'ensemble des statistiques du dataset.
    """
    import pandas as pd
    
    # Convertir en DataFrame
    df = pd.DataFrame(all_stats)
    
    # Afficher statistiques par classe
    print(f"\n{'='*70}")
    print("STATISTIQUES PAR CLASSE")
    print(f"{'='*70}")
    
    for class_label in sorted(df['class'].unique()):
        class_df = df[df['class'] == class_label]
        print(f"\nClasse {class_label}:")
        print(f"  Nombre d'images: {len(class_df)}")
        print(f"  Mean objet (avg): {class_df['mean_object'].mean():.4f} ± {class_df['mean_object'].std():.4f}")
        print(f"  Mean fond (avg): {class_df['mean_background'].mean():.4f} ± {class_df['mean_background'].std():.4f}")
    
    # Visualisation des distributions
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Distribution des intensités objet vs fond
    axes[0, 0].hist(df['mean_object'], bins=30, alpha=0.6, label='Objet', color='green')
    axes[0, 0].hist(df['mean_background'], bins=30, alpha=0.6, label='Fond', color='red')
    axes[0, 0].set_xlabel('Intensité moyenne')
    axes[0, 0].set_ylabel('Fréquence')
    axes[0, 0].set_title('Distribution des intensités moyennes')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Position des centres par classe
    for class_label in sorted(df['class'].unique()):
        class_df = df[df['class'] == class_label]
        centers_y = [c[0] for c in class_df['center']]
        centers_x = [c[1] for c in class_df['center']]
        axes[0, 1].scatter(centers_x, centers_y, label=f'Classe {class_label}', alpha=0.6, s=50)
    axes[0, 1].set_xlabel('X')
    axes[0, 1].set_ylabel('Y')
    axes[0, 1].set_title('Position des centres détectés')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    axes[0, 1].invert_yaxis()  # Y augmente vers le bas dans les images
    
    # Boxplot des intensités par classe
    data_object = [df[df['class'] == c]['mean_object'].values for c in sorted(df['class'].unique())]
    data_background = [df[df['class'] == c]['mean_background'].values for c in sorted(df['class'].unique())]
    
    positions_obj = np.arange(len(data_object)) * 2
    positions_bg = np.arange(len(data_background)) * 2 + 0.6
    
    bp1 = axes[1, 0].boxplot(data_object, positions=positions_obj, widths=0.5, 
                             patch_artist=True, boxprops=dict(facecolor='lightgreen'))
    bp2 = axes[1, 0].boxplot(data_background, positions=positions_bg, widths=0.5,
                             patch_artist=True, boxprops=dict(facecolor='lightcoral'))
    
    axes[1, 0].set_xticks(positions_obj + 0.3)
    axes[1, 0].set_xticklabels([f'Classe {c}' for c in sorted(df['class'].unique())])
    axes[1, 0].set_ylabel('Intensité moyenne')
    axes[1, 0].set_title('Intensités par classe')
    axes[1, 0].legend([bp1["boxes"][0], bp2["boxes"][0]], ['Objet', 'Fond'])
    axes[1, 0].grid(True, alpha=0.3)
    
    # Séparabilité objet/fond par image
    separability = (df['mean_object'] - df['mean_background']).abs()
    axes[1, 1].hist(separability, bins=30, color='purple', alpha=0.7)
    axes[1, 1].set_xlabel('|Mean Objet - Mean Fond|')
    axes[1, 1].set_ylabel('Fréquence')
    axes[1, 1].set_title('Séparabilité Objet/Fond')
    axes[1, 1].axvline(separability.mean(), color='red', linestyle='--', 
                      label=f'Moyenne: {separability.mean():.4f}')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join('visualizations', 'dataset_summary.png'), dpi=200, bbox_inches='tight')
    plt.show()
    
    print(f"\n✅ Graphique résumé sauvegardé: visualizations/dataset_summary.png")

In [ ]:
# 1. Visualiser TOUT le dataset
stats = visualize_all_dataset_sampling(
    data_dir='dataset_reflec',
    n_pixels_object=10000,
    n_pixels_background=5000,
    save_dir='visualizations',
    max_images=None  # None = toutes les images
)

# 2. Voir le résumé statistique
visualize_dataset_summary(stats)

# 3. Ouvrir le fichier HTML dans votre navigateur
# → visualizations/index.html

In [ ]:
# Visualiser seulement les 10 premières images
stats = visualize_all_dataset_sampling(
    data_dir='dataset_reflec',
    max_images=10,
    save_dir='visualizations_test'
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

def visualize_ghost_histograms(npy_file):
    """
    Visualise les histogrammes GHOST d'un fichier .npy.
    
    Args:
        npy_file: Chemin vers le fichier .npy contenant les histogrammes
    """
    # Charger les données
    data = np.load(npy_file)
    
    print(f"Fichier: {os.path.basename(npy_file)}")
    print(f"Shape: {data.shape}")
    print(f"Type: {data.dtype}")
    print(f"Min: {data.min():.6f}, Max: {data.max():.6f}")
    print(f"Mean: {data.mean():.6f}, Std: {data.std():.6f}")
    
    # Détecter le format
    if data.ndim == 3 and data.shape[0] == 1 and data.shape[1] == 1:
        # Format (1, 1, features) - aplatir en 1D
        data_1d = data.squeeze()
        print(f"\nFormat détecté: (1, 1, {len(data_1d)}) → Vecteur 1D")
        visualize_as_1d(data_1d, npy_file)
        
    elif data.ndim == 2:
        # Format (4, num_bins) ou (num_bins, 4)
        print(f"\nFormat détecté: Matrice 2D ({data.shape[0]} × {data.shape[1]})")
        
        # Détecter orientation
        if data.shape[0] == 4:
            # (4, num_bins) - histogrammes en lignes
            print("Orientation: 4 histogrammes en LIGNES")
            visualize_as_2d_rows(data, npy_file)
        elif data.shape[1] == 4:
            # (num_bins, 4) - histogrammes en colonnes
            print("Orientation: 4 histogrammes en COLONNES")
            visualize_as_2d_cols(data, npy_file)
        else:
            # Format inconnu, essayer les deux visualisations
            print("Orientation: Inconnue, affichage des deux possibilités")
            visualize_as_2d_both(data, npy_file)
    
    elif data.ndim == 1:
        # Vecteur 1D
        print(f"\nFormat détecté: Vecteur 1D ({len(data)} valeurs)")
        visualize_as_1d(data, npy_file)
    
    else:
        print(f"\n⚠️  Format non reconnu: {data.shape}")


def visualize_as_1d(data, npy_file):
    """
    Visualise un vecteur 1D (histogrammes concaténés).
    """
    n_features = len(data)
    
    # Deviner le nombre de bins par histogramme
    # Si n_features est divisible par 4, on a probablement 4 histogrammes
    if n_features % 4 == 0:
        num_bins = n_features // 4
        print(f"Déduction: 4 histogrammes × {num_bins} bins = {n_features} features")
        
        # Reshape en (4, num_bins)
        data_2d = data.reshape(4, num_bins)
        
        # Visualiser comme matrice 2D
        visualize_as_2d_rows(data_2d, npy_file)
    else:
        # Visualisation brute du vecteur
        fig, ax = plt.subplots(figsize=(16, 4))
        ax.plot(data, linewidth=1)
        ax.set_xlabel('Index', fontsize=12)
        ax.set_ylabel('Valeur', fontsize=12)
        ax.set_title(f'Vecteur 1D - {os.path.basename(npy_file)}\n{n_features} valeurs', 
                    fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()


def visualize_as_2d_rows(data, npy_file):
    """
    Visualise une matrice 2D avec histogrammes en LIGNES (4, num_bins).
    """
    n_hists, num_bins = data.shape
    
    hist_names = ['Magnitude (T)', 'Direction (θ)', 'Distance G', 'Distance W']
    
    if n_hists != 4:
        hist_names = [f'Histogramme {i+1}' for i in range(n_hists)]
    
    # Créer figure avec 3 types de visualisations
    fig = plt.figure(figsize=(20, 12))
    
    # === 1. Courbes séparées (4 subplots) ===
    for i in range(n_hists):
        ax = plt.subplot(4, 3, i*3 + 1)
        ax.plot(data[i], linewidth=2, color=f'C{i}')
        ax.fill_between(range(num_bins), data[i], alpha=0.3, color=f'C{i}')
        ax.set_ylabel('Densité', fontsize=10)
        ax.set_title(hist_names[i] if i < len(hist_names) else f'Hist {i+1}', 
                    fontsize=12, fontweight='bold')
        ax.grid(True, alpha=0.3)
        if i == n_hists - 1:
            ax.set_xlabel('Bin', fontsize=10)
    
    # === 2. Superposition ===
    ax_overlay = plt.subplot(1, 3, 2)
    for i in range(n_hists):
        ax_overlay.plot(data[i], linewidth=2, label=hist_names[i] if i < len(hist_names) else f'Hist {i+1}')
    ax_overlay.set_xlabel('Bin', fontsize=12)
    ax_overlay.set_ylabel('Densité', fontsize=12)
    ax_overlay.set_title('Superposition des 4 histogrammes', fontsize=14, fontweight='bold')
    ax_overlay.legend(fontsize=10)
    ax_overlay.grid(True, alpha=0.3)
    
    # === 3. Heatmap ===
    ax_heatmap = plt.subplot(1, 3, 3)
    im = ax_heatmap.imshow(data, aspect='auto', cmap='viridis', interpolation='nearest')
    ax_heatmap.set_xlabel('Bin', fontsize=12)
    ax_heatmap.set_ylabel('Histogramme', fontsize=12)
    ax_heatmap.set_title('Heatmap (Intensité = Densité)', fontsize=14, fontweight='bold')
    ax_heatmap.set_yticks(range(n_hists))
    ax_heatmap.set_yticklabels([hist_names[i] if i < len(hist_names) else f'H{i+1}' for i in range(n_hists)])
    plt.colorbar(im, ax=ax_heatmap, label='Densité')
    
    plt.suptitle(f'Visualisation GHOST - {os.path.basename(npy_file)}\nFormat: ({n_hists} histogrammes, {num_bins} bins chacun)', 
                 fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()
    
    # === 4. Statistiques ===
    print(f"\n{'='*70}")
    print("STATISTIQUES PAR HISTOGRAMME")
    print(f"{'='*70}")
    for i in range(n_hists):
        name = hist_names[i] if i < len(hist_names) else f'Histogramme {i+1}'
        print(f"\n{name}:")
        print(f"  Min:  {data[i].min():.6f}")
        print(f"  Max:  {data[i].max():.6f}")
        print(f"  Mean: {data[i].mean():.6f}")
        print(f"  Std:  {data[i].std():.6f}")
        print(f"  Sum:  {data[i].sum():.6f} (devrait ≈ 1.0 si normalisé)")


def visualize_as_2d_cols(data, npy_file):
    """
    Visualise une matrice 2D avec histogrammes en COLONNES (num_bins, 4).
    """
    # Transposer pour avoir histogrammes en lignes
    data_transposed = data.T
    visualize_as_2d_rows(data_transposed, npy_file)


def visualize_as_2d_both(data, npy_file):
    """
    Visualise les deux interprétations possibles (lignes et colonnes).
    """
    print("\n=== Interprétation 1: Histogrammes en LIGNES ===")
    visualize_as_2d_rows(data, npy_file)
    
    print("\n=== Interprétation 2: Histogrammes en COLONNES ===")
    visualize_as_2d_cols(data, npy_file)


def compare_multiple_files(npy_files, max_files=5):
    """
    Compare plusieurs fichiers GHOST côte à côte.
    
    Args:
        npy_files: Liste de chemins vers fichiers .npy
        max_files: Nombre maximum de fichiers à comparer
    """
    npy_files = npy_files[:max_files]
    
    n_files = len(npy_files)
    
    fig, axes = plt.subplots(n_files, 2, figsize=(16, 4*n_files))
    
    if n_files == 1:
        axes = axes.reshape(1, -1)
    
    for idx, npy_file in enumerate(npy_files):
        data = np.load(npy_file)
        
        # Détecter format et normaliser en (4, num_bins)
        if data.ndim == 3 and data.shape[0] == 1 and data.shape[1] == 1:
            data_1d = data.squeeze()
            if len(data_1d) % 4 == 0:
                num_bins = len(data_1d) // 4
                data_2d = data_1d.reshape(4, num_bins)
            else:
                continue
        elif data.ndim == 2:
            if data.shape[0] == 4:
                data_2d = data
            elif data.shape[1] == 4:
                data_2d = data.T
            else:
                continue
        elif data.ndim == 1 and len(data) % 4 == 0:
            num_bins = len(data) // 4
            data_2d = data.reshape(4, num_bins)
        else:
            continue
        
        # Extraire info du nom de fichier
        basename = os.path.basename(npy_file)
        try:
            class_label = basename.split('class')[1].split('.')[0]
        except:
            class_label = "?"
        
        # Subplot 1: Superposition
        hist_names = ['T', 'θ', 'G', 'W']
        for i in range(4):
            axes[idx, 0].plot(data_2d[i], linewidth=2, label=hist_names[i], alpha=0.8)
        axes[idx, 0].set_title(f'{basename}\nClasse: {class_label}', fontsize=11, fontweight='bold')
        axes[idx, 0].set_ylabel('Densité', fontsize=10)
        axes[idx, 0].legend(fontsize=9)
        axes[idx, 0].grid(True, alpha=0.3)
        
        # Subplot 2: Heatmap
        im = axes[idx, 1].imshow(data_2d, aspect='auto', cmap='viridis')
        axes[idx, 1].set_title(f'Heatmap - Classe {class_label}', fontsize=11, fontweight='bold')
        axes[idx, 1].set_ylabel('Histogramme', fontsize=10)
        axes[idx, 1].set_yticks(range(4))
        axes[idx, 1].set_yticklabels(hist_names)
        plt.colorbar(im, ax=axes[idx, 1])
    
    plt.suptitle('Comparaison de fichiers GHOST', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()


def explore_ghost_dataset(dataset_dir, max_display=5):
    """
    Explore un répertoire entier de fichiers GHOST.
    
    Args:
        dataset_dir: Répertoire contenant les .npy
        max_display: Nombre max de fichiers à afficher en détail
    """
    # Lister tous les fichiers
    npy_files = sorted([os.path.join(dataset_dir, f) 
                       for f in os.listdir(dataset_dir) 
                       if f.endswith('.npy')])
    
    print(f"{'='*70}")
    print(f"EXPLORATION DU DATASET GHOST")
    print(f"{'='*70}")
    print(f"Répertoire: {dataset_dir}")
    print(f"Fichiers trouvés: {len(npy_files)}")
    print(f"{'='*70}\n")
    
    # Analyser tous les fichiers
    shapes = []
    for npy_file in npy_files:
        data = np.load(npy_file)
        shapes.append(data.shape)
    
    # Résumé des shapes
    unique_shapes = list(set(shapes))
    print("Shapes trouvées:")
    for shape in unique_shapes:
        count = shapes.count(shape)
        print(f"  {shape}: {count} fichiers ({100*count/len(shapes):.1f}%)")
    
    print(f"\n{'='*70}")
    print(f"VISUALISATION DÉTAILLÉE ({max_display} premiers fichiers)")
    print(f"{'='*70}\n")
    
    # Visualiser quelques fichiers en détail
    for npy_file in npy_files[:max_display]:
        visualize_ghost_histograms(npy_file)
        print("\n" + "="*70 + "\n")
    
    # Comparaison côte à côte
    if len(npy_files) >= 2:
        print(f"\n{'='*70}")
        print("COMPARAISON CÔTE À CÔTE")
        print(f"{'='*70}\n")
        compare_multiple_files(npy_files, max_files=min(5, len(npy_files)))

In [ ]:
visualize_ghost_histograms('dataset_hist/2025-04-03-G1_00-1_hist_class0.npy')


In [ ]:
# utiliser matplotlib  pourafficher les valeur de la matrice 2D des histogrammes 
matrice = np.load('dataset_hist/2025-04-03-G1_00-1_hist_class0.npy')

fig, ax = plt.subplots(figsize=(12, 8))
im = ax.imshow(matrice, cmap='YlGn') # Jaune à Vert

# Afficher les valeurs numériques dans chaque case
for i in range(len(matrice)):
    for j in range(len(matrice[0])):
        text = ax.text(j, i, round(matrice[i, j], 2),
                       ha="center", va="center", color="black")

plt.title("Visualisation d'une matrice")
plt.show()

In [ ]:
matrice = np.load('dataset_hist/2025-05-18-G4_20_hist_class3.npy')

fig, ax = plt.subplots(figsize=(12, 8))
im = ax.imshow(matrice, cmap='YlGn') # Jaune à Vert

# Afficher les valeurs numériques dans chaque case
for i in range(len(matrice)):
    for j in range(len(matrice[0])):
        text = ax.text(j, i, round(matrice[i, j], 2),
                       ha="center", va="center", color="black")

plt.title("Visualisation d'une matrice")
plt.show()

In [ ]:
matrice = np.load('dataset_hist/2025-05-06-G3_19_hist_class2.npy')

fig, ax = plt.subplots(figsize=(12, 8))
im = ax.imshow(matrice, cmap='YlGn') # Jaune à Vert

# Afficher les valeurs numériques dans chaque case
for i in range(len(matrice)):
    for j in range(len(matrice[0])):
        text = ax.text(j, i, round(matrice[i, j], 2),
                       ha="center", va="center", color="black")

plt.title("Visualisation d'une matrice")
plt.show()